In [7]:
import pandas as pd
import sqlite3 as sql

In [8]:
def create_db(db):
    """
        Función que crea una base de datos
    """
    
    conn = sql.connect(db)
    conn.commit()

In [9]:
def table_db(db, query):
    """
        Función que crea una tabla en la base de datos.
    """    
    conn = sql.connect(db)
    cursor = conn.cursor()
    
    cursor.execute(query)
    
    conn.commit()
    conn.close()

In [10]:
def insertar_filas(db, query, data):
    """
        Función que inserta registros a la tabla creada.
    """
    
    conn = sql.connect(db)
    cursor = conn.cursor()
    
    cursor.executemany(query, data)
    
    conn.commit()
    conn.close()    

In [11]:
def selectSQL(db, query):
    """
        Función que ejecuta querys en la base de datos.
    """
    
    conn = sql.connect(db)
    cursor = conn.cursor()
    
    cursor.execute(query)
    
    datos = cursor.fetchall()
    
    conn.commit()
    conn.close()
    
    return datos

In [12]:
# Se importa la base de datos:
data_df = pd.read_csv("posibles_clientes.csv", header = 0, sep = ",")

In [13]:
# Se transforma la base de datos de dataframe a lista
data_list = data_df.values.tolist()

In [14]:
# Se crea la base de datos y la tabla:
db = "Movired.db"

query = """CREATE TABLE posibles_usuarios(
            documento INT,
            nombre TEXT,
            CELULAR INT,
            TELEFONO INT);"""

create_db(db)

table_db(db, query)

In [15]:
# Se añaden los registros a la tabla creada:
query_1 = f"INSERT INTO posibles_usuarios VALUES(?,?,?,?);"
insertar_filas(db, query_1, data_list)

In [16]:
# Se realizan consultas a la base de datos.

query_2 = "SELECT SUM(CASE WHEN LENGTH(documento) < 9 THEN 1 ELSE 0 END) AS personas_mas_40_anhos, \
SUM(CASE WHEN LENGTH(documento) >= 9 THEN 1 ELSE 0 END) AS personas_40_menos_anhos \
FROM posibles_usuarios;"

sql_result = selectSQL(db, query_2)

print(f"Personas que posiblemente son mayores de 40 años: {sql_result[0][0]}")
print(f"Personas que posiblemente tienen 40 años o menos: {sql_result[0][1]}")

Personas que posiblemente son mayores de 40 años: 891
Personas que posiblemente tienen 40 años o menos: 51


In [17]:
# Se realiza un update a la base de datos.

num_doc = 72435
nombre_update = "Francisco Javier G L"
# ------------------------------------------------------------------------------------------------
query_3 = f"SELECT * FROM posibles_usuarios WHERE documento = {num_doc}"
sql_result = selectSQL(db, query_3)

print(sql_result)
# ------------------------------------------------------------------------------------------------
query_4 = f"UPDATE posibles_usuarios SET nombre = '{nombre_update}' WHERE documento = {num_doc}"
sql_result = selectSQL(db, query_4)
# ------------------------------------------------------------------------------------------------
query_5 = f"SELECT * FROM posibles_usuarios WHERE documento = {num_doc}"
sql_result = selectSQL(db, query_5)

print(sql_result)

[(72435, 'PACIFICO  CUBILLOS VILLAMARIN', 3166294231, 4022971), (72435, 'PACIFICO  CUBILLOS VILLAMARIN', 3177355230, 5548250)]
[(72435, 'Francisco Javier G L', 3166294231, 4022971), (72435, 'Francisco Javier G L', 3177355230, 5548250)]


In [18]:
# Se eliminan registros de la base de datos.
num_doc = 72435
# ------------------------------------------------------------------------------------------------
query_6 = "SELECT count(*) FROM posibles_usuarios"
sql_result = selectSQL(db, query_6)

print(sql_result)
# ------------------------------------------------------------------------------------------------
query_7 = f"DELETE FROM posibles_usuarios WHERE documento = {num_doc}"
sql_result = selectSQL(db, query_7)
# ------------------------------------------------------------------------------------------------
query_8 = "SELECT count(*) FROM posibles_usuarios"
sql_result = selectSQL(db, query_8)

print(sql_result)

[(942,)]
[(940,)]


In [19]:
# Se elimina la tabla:
query_9 = "DROP TABLE posibles_usuarios"
sql_result = selectSQL(db, query_9)

In [20]:
# Truquito para crear archivo .sql
import csv

expresion = "INSERT INTO movired.posibles_usuarios (documento, nombre, celular, telefono) VALUES ("

with open('posibles_clientes_query.csv', 'w', newline='') as student_file:
    writer = csv.writer(student_file)
    
    for item in data_list:
    
        row = expresion + str(item[0]) + ",'" + str(item[1]) + "'," + str(item[2]) + "," + str(item[3]) + ");"
 
        writer.writerow([row])